In [19]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Numeric, DateTime
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.automap  import automap_base

In [20]:
# The code was removed by Watson Studio for sharing.

In [21]:

engine = create_engine(connection_string, echo=False)
meta = MetaData()

nome_tabela = 'COVID-BR'

covid = Table('COVID-BR', meta, 
    Column("COUNTRY", String(30)),
    Column("PROVINCE_STATE", String(50)),
    Column("CITY", String(50)),
    Column("IBGEID", Integer),
    Column("DEATHS", Integer),
    Column("CASES", Integer),
    Column("DEATHS_PER_100K_INHABITANTS", Numeric(18,8)),
    Column("TOTALCASES_PER_100K_INHABITANTS", Numeric(18,8)),
    Column("DEATHS_BY_TOTALCASES", Numeric(18,8)),
    Column("SOURCE", String(10)),
    Column("DATE", Date),
    Column("NEWCASES", Integer),
    Column("NEWDEATHS",Integer))

# date	newCases	newDeaths
#'DEATHS_PER_100K_INHABITANTS','TOTALCASES_PER_100K_INHABITANTS','DEATHS_BY_TOTALCASES'

dropTable = True

if not engine.dialect.has_table(engine, nome_tabela, schema = 'BLUADMIN'):
    meta.create_all(engine)
else:
    if dropTable:
        covid.drop(engine) # Apaga tabela
        meta.create_all(engine) # Cria tabela
    conn = engine.connect()
    #stmt = covid.delete().where(students.c.lastname == 'Khanna')
    stmt = covid.delete()
    conn.execute(stmt)
    conn.close()

In [22]:
Base = automap_base()
# connection_string = "db2+ibm_db://user:pass@host[:port]/database"
#connection_string = "db2+ibm_db://tfn53947:9pwx9bjsmt^grb8h@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB"

#engine = create_engine(connection_string, echo=True)
session = scoped_session(sessionmaker(autocommit=True,
                                      autoflush=False,
                                      bind=engine))

Base.prepare(engine, reflect=True)
base_sa = Base.classes

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities.csv')
df.head()

,country,state,city,ibgeID,deaths,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source,date,newCases,newDeaths
0,Brazil,MG,Abadia dos Dourados/MG,3100104,0,5,0.00000,71.54099,0.00000,MS,2020-05-20,0,0
1,Brazil,PA,Abaetetuba/PA,1500107,15,467,9.51185,296.13565,0.03212,MS,2020-05-20,25,0
2,Brazil,MG,Abaeté/MG,3100203,0,1,0.00000,4.30348,0.00000,MS,2020-05-20,0,0
3,Brazil,CE,Abaiara/CE,2300101,0,1,0.00000,8.52006,0.00000,MS,2020-05-20,0,0
4,Brazil,BA,Abaré/BA,2900207,0,0,0.00000,0.00000,0.00000,MS,2020-05-20,0,0


In [24]:
df.columns = ['COUNTRY','PROVINCE_STATE','CITY',
                     'IBGEID','DEATHS','CASES','DEATHS_PER_100K_INHABITANTS',
              'TOTALCASES_PER_100K_INHABITANTS','DEATHS_BY_TOTALCASES','SOURCE','DATE',
              'NEWCASES','NEWDEATHS']
df.head()

,COUNTRY,PROVINCE_STATE,CITY,IBGEID,DEATHS,CASES,DEATHS_PER_100K_INHABITANTS,TOTALCASES_PER_100K_INHABITANTS,DEATHS_BY_TOTALCASES,SOURCE,DATE,NEWCASES,NEWDEATHS
0,Brazil,MG,Abadia dos Dourados/MG,3100104,0,5,0.00000,71.54099,0.00000,MS,2020-05-20,0,0
1,Brazil,PA,Abaetetuba/PA,1500107,15,467,9.51185,296.13565,0.03212,MS,2020-05-20,25,0
2,Brazil,MG,Abaeté/MG,3100203,0,1,0.00000,4.30348,0.00000,MS,2020-05-20,0,0
3,Brazil,CE,Abaiara/CE,2300101,0,1,0.00000,8.52006,0.00000,MS,2020-05-20,0,0
4,Brazil,BA,Abaré/BA,2900207,0,0,0.00000,0.00000,0.00000,MS,2020-05-20,0,0


In [25]:
nome_tabela = 'COVID-BR'

In [26]:
df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False)
#df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False, chunksize=1000)   # With number of records to be processed each time

In [27]:
df2 = pd.read_sql_table(nome_tabela, engine)
df2.head()

,country,province_state,city,ibgeid,deaths,cases,deaths_per_100k_inhabitants,totalcases_per_100k_inhabitants,deaths_by_totalcases,SOURCE,DATE,newcases,newdeaths
0,Brazil,MG,Abadia dos Dourados/MG,3100104,0,5,0.00000,71.54099,0.00000,MS,2020-05-20,0,0
1,Brazil,PA,Abaetetuba/PA,1500107,15,467,9.51185,296.13565,0.03212,MS,2020-05-20,25,0
2,Brazil,MG,Abaeté/MG,3100203,0,1,0.00000,4.30348,0.00000,MS,2020-05-20,0,0
3,Brazil,CE,Abaiara/CE,2300101,0,1,0.00000,8.52006,0.00000,MS,2020-05-20,0,0
4,Brazil,BA,Abaré/BA,2900207,0,0,0.00000,0.00000,0.00000,MS,2020-05-20,0,0


In [28]:
session.close()